In [2]:
from math import log10, pow, sqrt, pi

#### расстояние до базовых станций:
- ha – высота антены,
- hm – твоя высота.
- d - расстояние между

In [3]:
def d3d(ha, hm, d):
    if ha >= hm:
        return sqrt(pow((ha - hm), 2) + pow(d, 2))
    else:
        return sqrt(pow((hm - ha), 2) + pow(d, 2))

In [4]:
d_list = [155, 152, 158]
ha = 38
hm = 17

dist3d = []

for d in d_list:
    dist3d.append(d3d(ha, hm, d))


dist3d

[156.41611170208776, 153.44380078712857, 159.38946012832844]

#### Переводы единиц

In [5]:
def db_to_dbm(db):
    return db + 30

def dbm_to_db(db):
    return db - 30

#### 2G
Формула - *COST–Hata–Model*

Параметры:
- hte – эффективная высота базовой антенны в метрах (от 30 до 200 м)
- hre – эффективная высота мобильной антенны в метрах (от 1 до 10 м),
- fc – частота сигнала в МГц от 150 до 1500 МГц
- d – расстояние от передатчика до приемника, км

_a(hre)_ – корректирующий фактор для эффективной высоты
мобильной антенны, который является функцией величины зоны
обслуживания. Используется версия формулы для крупных городов (1.12) в методичке

In [6]:
def PLg2(fc, hte, hre, d, Cm):
    return 46.3  + 33.9 * log10(fc) - 13.82 * log10(hte) - a(hre)\
           + (44.9 - 6.55 * log10(hte)) * log10(d) + Cm

def a(hre):
    return 3.2 * pow((log10(11.75 * hre)), 2) - 4.97

Вычисление затухания

In [7]:

for (i, dst) in enumerate(dist3d, start=1):
    print(f"2G {i}:",
           43 - db_to_dbm(PLg2(fc=960, hte=30, hre=10, d= dst / 1000, Cm=3) - 15), "dBm")


2G 1: -64.86167893160236 dBm
2G 2: -64.56818056129211 dBm
2G 3: -65.14975181514485 dBm


#### 3G
- d – расстояние между передатчиком и приемником (м),
- n – количество этажей на пути.

In [8]:
def PLg3(n, d):
    return 37 + 30 * log10(d) + 18.3 * pow(n, ((n + 2) / (n + 1) - 0.46))

Расчет

In [9]:

for (i, dst) in enumerate(dist3d, start=1):
    print(f"3G {i}:",
           43 - db_to_dbm(PLg3(n=0, d=dst) - 15), "dBm")
    

3G 1: -74.82844457097113 dBm
3G 2: -74.57848042168112 dBm
3G 3: -75.07378798869472 dBm


#### 4G
- fc – частота сигнала в ГГц
- h – средняя высота зданий в диапазоне 5 м < h < 50 м, типовое значение h = 20 м,
- W – ширина улицы в диапазоне 5 м < W < 50 м, типовое значение W = 20 м
- hBS и hUT – действительные высоты антенн
- типовое значение hBS = 25 м и 10 м < hBS < 150 м,

In [10]:
def PLg4(W, h, hBS, d3D, fc, hUT):
    return 161.04 - 7.1 * log10(W) + 7.5 * log10(h)- (24.37 - 3.7* pow((h / hBS),2)) * log10(hBS)\
           + (43.42 - 3.1 * log10(hBS)) * (log10(d3D) - 3) + 20 * log10(fc)\
           - (3.2 * pow(log10(17.625),2) - 4.97) - 0.6 * (hUT - 1.5)


Расчет

In [11]:
for (i, dst) in enumerate(dist3d, start=1):
    print(f"4G {i}:",
           43 - db_to_dbm(PLg4(W=20, h=25, hBS=25, d3D=dst, fc=2.6, hUT=17.5) - 15), "dBm")
    # print(PLg4(W=20, h=20, hBS=25, d3D=155, fc=2.6, hUT=22.5))

4G 1: -72.59957828729232 dBm
4G 2: -72.27390511333927 dBm
4G 3: -72.91923120466257 dBm


#### WiFi

In [12]:
def PLwf(d, fc):
    return - 20 * log10 (lm(fc) / (4 * pi * d))

def lm(fc):
    return 299_792_458 / fc

Расчет
- Мощность передатчика точки доступа WiFi 20 дБм
-  Коэффициент усиления антенны точки доступа и WiFi
телефона 0 дБ

In [14]:
fc = 5_000_000_000
wifi_dist = [0.5, 6, 2.5]



for (i, dst) in enumerate(wifi_dist, start=1):
    print(f"Wifi {i}:", 20 - db_to_dbm(PLwf(dst, fc) + 3), "dBm")



Wifi 1: -53.406583395324134 dBm
Wifi 2: -74.99020831627664 dBm
Wifi 3: -67.38598348204451 dBm
